In [1]:
import sys
sys.path.insert(1, '/Users/brian/crypto/crocket')

In [2]:
from json import load as json_load
from random import randint, shuffle
from requests import get
from time import sleep, time
from requests_futures.sessions import FuturesSession
from requests.exceptions import ConnectTimeout, ProxyError

from crocket.bittrex.bittrex2 import Bittrex, return_request_input

In [3]:
PROXY_LIST_PATH = '/Users/brian/crypto/crocket/proxy_list.txt'
BITTREX_CREDENTIALS_PATH = '/Users/brian/bittrex_credentials.json'

In [4]:
def configure_ip(ip):
    
    return {
        'http': ip,
        'https': ip
    }

def process_response(session, response):
    
    try:
        response.data = response.json()
    except Exception as e:
        print(e)
        response.data = {
               'success': False,
               'message': 'NO_API_RESPONSE',
               'result': None
            }

In [5]:
# Read files

with open(PROXY_LIST_PATH, 'r') as f:
    proxies = f.read().splitlines()
    
with open(BITTREX_CREDENTIALS_PATH, 'r') as f:
    credentials = json_load(f)

In [6]:
# Create bittrex objects

b1 = Bittrex(api_key=credentials.get('key'), 
             api_secret=credentials.get('secret'), 
             api_version='v2.0')

b2 = Bittrex(api_key=credentials.get('key'), 
             api_secret=credentials.get('secret'), 
             dispatch=return_request_input,
             api_version='v2.0')

In [7]:
# Get list of currencies

response = b1.get_markets()

currencies = [x.get('MarketName') for x in response.get('result') 
              if x.get('BaseCurrency') == 'BTC' and x.get('IsActive')]

markets = currencies

In [19]:
rind = list(range(200))

In [39]:
# Test asynchronous requests

with FuturesSession(max_workers=200) as session:
    
    for ii in range(0, 4): # TODO: replace with while loop
        
        shuffle(rind)
        
        inputs = []
        responses1 = []
        
        start = time()

        for index in range(len(markets)):

            request_input = b2.get_market_history(markets[index])
            inputs.append(request_input)
            
            proxy = configure_ip(proxies[rind[index]])

            future = session.get(request_input.get('url'), 
                                 background_callback=process_response,
                                 headers={"apisign": request_input.get('apisign')},
                                 timeout=5,
                                 proxies=proxy)

            responses1.append([markets[index], future])
            print('IP: {}. Elapsed time: {}s'.format(proxies[rind[index]], str(s2-s1)))

        stop = time()
        print('Total elapsed time: {}s'.format(str(stop-start)))
        
        
        for index in range(len(markets)):
            try:
                responses1[index][1] = responses1[index][1].result()
            except (ProxyError, ConnectTimeout):
                print('Failed API call for {}, index: {}.'.format(markets[index], str(index)))
                
                proxy = configure_ip(proxies[rind[index]])
                future = session.get(inputs[index].get('url'), 
                                 background_callback=process_response,
                                 headers={"apisign": inputs[index].get('apisign')},
                                 timeout=5,
                                 proxies=proxy)
                
                responses1[index][1] = responses1[index][1].result()
                
                
        print(responses1)
        
        sleep(30)

IP: 173.234.181.224:8800. Elapsed time: 0.013106107711791992s
IP: 173.234.181.90:8800. Elapsed time: 0.010975837707519531s
IP: 192.161.163.114:8800. Elapsed time: 0.002855062484741211s
IP: 173.234.59.146:8800. Elapsed time: 0.015824079513549805s
IP: 192.126.161.76:8800. Elapsed time: 0.007608890533447266s
IP: 173.234.59.177:8800. Elapsed time: 0.00975489616394043s
IP: 173.208.46.129:8800. Elapsed time: 0.0006709098815917969s
IP: 173.208.103.120:8800. Elapsed time: 0.006524801254272461s
IP: 173.234.165.90:8800. Elapsed time: 0.0030868053436279297s
IP: 173.208.103.83:8800. Elapsed time: 0.0012440681457519531s
IP: 23.19.34.105:8800. Elapsed time: 0.0035860538482666016s
IP: 173.234.165.246:8800. Elapsed time: 0.003255128860473633s
IP: 173.234.59.68:8800. Elapsed time: 0.006067037582397461s
IP: 173.234.59.45:8800. Elapsed time: 0.0007860660552978516s
IP: 192.161.163.103:8800. Elapsed time: 0.002685070037841797s
IP: 192.126.157.100:8800. Elapsed time: 0.0042231082916259766s
IP: 192.126.164.1

ProxyError: HTTPSConnectionPool(host='bittrex.com', port=443): Max retries exceeded with url: /api/v2.0/pub/Market/GetMarketHistory?marketname=BTC-DASH&market=BTC-DASH (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))

In [37]:
with FuturesSession(max_workers=200) as session:
    future = session.get(inputs[5].get('url'), 
                                     background_callback=process_response,
                                     headers={"apisign": inputs[5].get('apisign')},
                                     timeout=10,
                                     proxies=configure_ip(proxies[rind[5]]))

In [38]:
future

<Future at 0x13a8b0ac8 state=finished returned Response>

In [40]:
responses1

[['BTC-LTC', <Response [200]>],
 ['BTC-DOGE', <Response [200]>],
 ['BTC-VTC', <Response [200]>],
 ['BTC-PPC', <Response [200]>],
 ['BTC-FTC', <Response [200]>],
 ['BTC-RDD', <Response [200]>],
 ['BTC-NXT', <Response [200]>],
 ['BTC-DASH', <Future at 0x1130afa20 state=finished raised ProxyError>],
 ['BTC-POT', <Future at 0x1130ba048 state=finished raised ProxyError>],
 ['BTC-BLK', <Future at 0x1130ba588 state=finished raised ProxyError>],
 ['BTC-EMC2', <Future at 0x1130bac88 state=finished returned Response>],
 ['BTC-XMY', <Future at 0x1130c05f8 state=finished raised ProxyError>],
 ['BTC-AUR', <Future at 0x1130c0b00 state=finished returned Response>],
 ['BTC-EFL', <Future at 0x1130c54a8 state=finished returned Response>],
 ['BTC-GLD', <Future at 0x1130c59e8 state=finished returned Response>],
 ['BTC-SLR', <Future at 0x1130c5b70 state=finished returned Response>],
 ['BTC-PTC', <Future at 0x1130afd30 state=finished returned Response>],
 ['BTC-GRS', <Future at 0x1130c9c50 state=finished re